# ⚠️ IMPORTANT: Setup Requirements

## 🛠️ Prerequisites Before Running This Notebook

### ✅ Already Installed:
- ✅ PyTorch 2.6.0 with CUDA 12.4
- ✅ Virtual environment (`.venv`)
- ✅ MambaTSR repository cloned

### ⏳ Still Need to Install:

#### 1. Microsoft Visual C++ Build Tools (Required!)
**Why needed:** To compile CUDA kernels for selective scan operation

**Download:** https://visualstudio.microsoft.com/visual-cpp-build-tools/

**Install workloads:**
- ✅ Desktop development with C++
- ✅ MSVC v143 - VS 2022 C++ x64/x86 build tools
- ✅ Windows 10 SDK

**Time needed:** ~30-40 minutes (download + install + compile)

#### 2. Compile selective_scan kernel
After installing Build Tools, run in terminal:

```powershell
cd G:\Dataset\MambaTSR\kernels\selective_scan
pip install --no-build-isolation -e .
```

---

## 📋 Full Setup Guide

**See:** `G:\Dataset\MAMBATSR_SETUP_GUIDE.md` for detailed instructions

---

## ⚡ Quick Start (After Setup)

Once Build Tools is installed and selective_scan is compiled:
1. Restart VS Code
2. Select kernel: `.venv` (Python 3.11)
3. Run all cells below

---

**Current Status:** ⏳ Waiting for Build Tools installation

# MambaTSR for PlantVillage Disease Classification
## Áp dụng kiến trúc MambaTSR (State Space Model) để nhận dạng bệnh cây

**Model**: Super_Mamba với Vision State Space (VSS) Blocks  
**Dataset**: PlantVillage (39 classes)  
**Paper**: "MambaTSR: You Only Need 90k Parameters for Traffic Sign Recognition" (Neurocomputing, JCR Q1)  

### Kiến trúc chính:
- **ConvNet**: Embedding ban đầu từ ảnh RGB
- **PatchMerging2D + VSSBlock**: 6 tầng (depth=6)
- **SS2D (Selective Scan 2D)**: Trái tim của Mamba architecture
- **Classifier**: LayerNorm → AvgPool → Linear(num_classes=39)

## 1. Setup Environment & Dependencies

In [ ]:
# Import standard libraries
import os
import sys
import time
import math
import copy
import random
import logging
from functools import partial
from typing import Optional, Callable, Any
from collections import OrderedDict
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau

# Import torchvision
from torchvision import datasets, transforms
from torchvision.ops import Permute

# Import timm
import timm
from timm.models.layers import DropPath, trunc_normal_

# Import other utilities
from einops import rearrange, repeat
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. Add MambaTSR to Python Path

In [ ]:
# Add MambaTSR models to path
mamba_path = Path(r'G:\Dataset\MambaTSR')
if str(mamba_path) not in sys.path:
    sys.path.insert(0, str(mamba_path))
    print(f"✓ Added {mamba_path} to sys.path")

# Verify path
print("\nPython search paths:")
for p in sys.path[:3]:
    print(f"  - {p}")

## 3. Import MambaTSR Components

**Quan trọng**: Đây là phần thầy yêu cầu - copy các components từ MambaTSR

In [ ]:
# Import MambaTSR core components
try:
    from models.ConvNet import ConvNet
    from models.VSSBlock import VSSBlock
    from models.vmamba import SS2D, Mlp
    print("✓ Successfully imported MambaTSR components")
except ImportError as e:
    print(f"❌ Error importing MambaTSR components: {e}")
    print("\nĐảm bảo đã cài đặt selective_scan CUDA kernel:")
    print("cd G:\\Dataset\\MambaTSR\\kernels\\selective_scan")
    print("pip install .")
    raise

## 4. Define PatchMerging2D

Component từ VSSBlock_utils.py - giảm spatial dimension và tăng channels

In [ ]:
class PatchMerging2D(nn.Module):
    """Patch Merging Layer - giảm H, W xuống 1/2 và tăng channels"""
    def __init__(self, dim, out_dim=-1, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, (2 * dim) if out_dim < 0 else out_dim, bias=False)
        self.norm = norm_layer(4 * dim)

    @staticmethod
    def _patch_merging_pad(x: torch.Tensor):
        H, W, _ = x.shape[-3:]
        if (W % 2 != 0) or (H % 2 != 0):
            x = F.pad(x, (0, 0, 0, W % 2, 0, H % 2))
        x0 = x[..., 0::2, 0::2, :]  # ... H/2 W/2 C
        x1 = x[..., 1::2, 0::2, :]  # ... H/2 W/2 C
        x2 = x[..., 0::2, 1::2, :]  # ... H/2 W/2 C
        x3 = x[..., 1::2, 1::2, :]  # ... H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # ... H/2 W/2 4*C
        return x

    def forward(self, x):
        x = self._patch_merging_pad(x)
        x = self.norm(x)
        x = self.reduction(x)
        return x

print("✓ PatchMerging2D defined")

## 5. Define Super_Mamba Model

**Đây là dòng 59 mà thầy yêu cầu** - Class Super_Mamba từ VSSBlock_utils.py  
**Chỉnh sửa**: `num_classes=39` cho PlantVillage (thay vì 43 cho traffic signs)

In [ ]:
class PermuteLayer(nn.Module):
    """Permute layer for dimension reordering"""
    def __init__(self, *args):
        super().__init__()
        self.args = args

    def forward(self, x: torch.Tensor):
        return x.permute(*self.args)


class Super_Mamba(nn.Module):
    """Super_Mamba Model cho PlantVillage Classification
    
    Kiến trúc:
    - ConvNet: Feature embedding ban đầu
    - 6 stages: PatchMerging2D + VSSBlock (Selective Scan 2D)
    - Classifier: LayerNorm → Permute → AvgPool → Linear
    
    Args:
        dims: Số channels ban đầu (default=3 cho RGB)
        depth: Số tầng VSSBlock (default=6)
        num_classes: Số classes output (39 cho PlantVillage)
    """
    def __init__(self, dims=3, depth=6, num_classes=39):
        super().__init__()
        self.depth = depth
        self.preembd = ConvNet()  # Embedding layer
        
        # Calculate dimensions for each layer
        if isinstance(dims, int):
            dims = [int(dims * 2 ** i_layer) for i_layer in range(self.depth+1)]
        self.num_features = dims[-1]
        self.dims = dims
        
        # Build layers: PatchMerging + VSSBlock repeated depth times
        self.layers = nn.ModuleList()
        for i_layer in range(self.depth):
            downsample = PatchMerging2D(
                self.dims[i_layer],
                self.dims[i_layer + 1],
                norm_layer=nn.LayerNorm,
            )
            vss_block = VSSBlock(hidden_dim=self.dims[i_layer+1])
            self.layers.append(downsample)
            self.layers.append(vss_block)

        # Classifier head
        self.classifier = nn.Sequential(OrderedDict(
            norm=nn.LayerNorm(self.num_features),  # B,H,W,C
            permute=PermuteLayer(0, 3, 1, 2),
            avgpool=nn.AdaptiveAvgPool2d(1),
            flatten=nn.Flatten(1),
            head=nn.Linear(self.num_features, num_classes),
        ))

        self.apply(self._init_weights)

    def _init_weights(self, m: nn.Module):
        """Initialize weights"""
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        x = self.preembd(x)  # ConvNet embedding
        x = x.permute(0, 2, 3, 1)  # [B, C, H, W] -> [B, H, W, C]
        for layers in self.layers:
            x = layers(x)
        x = self.classifier(x)
        return x

print("✓ Super_Mamba model defined với num_classes=39 cho PlantVillage")

## 6. Configuration & Hyperparameters

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Hyperparameters
CONFIG = {
    # Model
    'model_name': 'Super_Mamba',
    'dims': 3,  # Initial channels
    'depth': 6,  # Number of VSSBlock stages
    'num_classes': 39,  # PlantVillage classes
    
    # Training
    'batch_size': 64,
    'num_epochs': 100,
    'learning_rate': 1e-3,
    'weight_decay': 1e-4,
    'num_workers': 0,  # Windows compatibility
    'pin_memory': True if torch.cuda.is_available() else False,
    
    # Data augmentation
    'image_size': 32,  # MambaTSR sử dụng 32x32
    'brightness': 0.8,
    'contrast': (1.0, 1.0),
    
    # Learning rate scheduler
    'scheduler': 'cosine',  # 'cosine' or 'plateau'
    'min_lr': 1e-6,
    
    # Early stopping
    'patience': 15,
    
    # Paths
    'data_root': Path(r'G:\Dataset\Data\PlantVillage\PlantVillage-Dataset-master'),
    'save_dir': Path(r'G:\Dataset\models\MambaTSR'),
}

# Create save directory
CONFIG['save_dir'].mkdir(parents=True, exist_ok=True)
print(f"\n✓ Models will be saved to: {CONFIG['save_dir']}")
print(f"\n📋 Configuration:")
for key, value in CONFIG.items():
    if not isinstance(value, Path):
        print(f"  {key}: {value}")

## 7. Data Preparation

In [ ]:
# Data transforms - theo cấu trúc của MambaTSR
transform_train = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    transforms.ColorJitter(brightness=CONFIG['brightness'], contrast=CONFIG['contrast']),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_val = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

print("✓ Data transforms defined")
print(f"  - Train: ColorJitter + Flip + Rotation + Normalize")
print(f"  - Val/Test: Resize + Normalize")

In [ ]:
# Load datasets
print("Loading PlantVillage dataset...")

full_dataset = datasets.ImageFolder(root=CONFIG['data_root'])
class_names = full_dataset.classes
num_classes = len(class_names)

print(f"✓ Found {num_classes} classes: {class_names[:5]}...")
print(f"✓ Total images: {len(full_dataset):,}")

# Split dataset: 72% train, 18% val, 10% test
total_size = len(full_dataset)
train_size = int(0.72 * total_size)
val_size = int(0.18 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Apply transforms
train_dataset.dataset.transform = transform_train
val_dataset.dataset.transform = transform_val
test_dataset.dataset.transform = transform_val

print(f"\n📊 Dataset split:")
print(f"  - Train: {len(train_dataset):,} images ({len(train_dataset)/total_size*100:.1f}%)")
print(f"  - Val:   {len(val_dataset):,} images ({len(val_dataset)/total_size*100:.1f}%)")
print(f"  - Test:  {len(test_dataset):,} images ({len(test_dataset)/total_size*100:.1f}%)")

In [ ]:
# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory']
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory']
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory']
)

print("\n✓ DataLoaders created")
print(f"  - Train batches: {len(train_loader)}")
print(f"  - Val batches: {len(val_loader)}")
print(f"  - Test batches: {len(test_loader)}")

## 8. DataLoader Smoke Test

In [ ]:
# Test DataLoader
print("Testing DataLoader...")
test_batch = next(iter(train_loader))
images, labels = test_batch

print(f"✓ Batch shape: {images.shape}")
print(f"✓ Labels shape: {labels.shape}")
print(f"✓ Image range: [{images.min():.3f}, {images.max():.3f}]")
print(f"✓ Device: {images.device}")

if torch.cuda.is_available():
    images_gpu = images.to(device)
    print(f"✓ Successfully moved batch to GPU: {images_gpu.device}")

## 9. Initialize Model

In [ ]:
# Create Super_Mamba model
print("Initializing Super_Mamba model...")
model = Super_Mamba(
    dims=CONFIG['dims'],
    depth=CONFIG['depth'],
    num_classes=CONFIG['num_classes']
).to(device)

print(f"✓ Model created and moved to {device}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Model Statistics:")
print(f"  - Total parameters: {total_params:,}")
print(f"  - Trainable parameters: {trainable_params:,}")
print(f"  - Model size: {total_params * 4 / 1024**2:.2f} MB (float32)")

In [ ]:
# Test forward pass
print("\nTesting forward pass...")
model.eval()
with torch.no_grad():
    test_input = torch.randn(2, 3, CONFIG['image_size'], CONFIG['image_size']).to(device)
    test_output = model(test_input)
    print(f"✓ Input shape: {test_input.shape}")
    print(f"✓ Output shape: {test_output.shape}")
    print(f"✓ Output range: [{test_output.min():.3f}, {test_output.max():.3f}]")

model.train()
print("✓ Forward pass successful!")

## 10. Training Setup

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer - theo paper MambaTSR
optimizer = AdamW(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

# Learning rate scheduler
if CONFIG['scheduler'] == 'cosine':
    scheduler = CosineAnnealingLR(
        optimizer,
        T_max=CONFIG['num_epochs'],
        eta_min=CONFIG['min_lr']
    )
    print("✓ Using CosineAnnealingLR scheduler")
else:
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.5,
        patience=5,
        min_lr=CONFIG['min_lr']
    )
    print("✓ Using ReduceLROnPlateau scheduler")

print(f"✓ Optimizer: AdamW (lr={CONFIG['learning_rate']}, weight_decay={CONFIG['weight_decay']})")
print(f"✓ Loss function: CrossEntropyLoss")

## 11. Training Functions

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]")
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f"{running_loss/(batch_idx+1):.4f}",
            'acc': f"{100.*correct/total:.2f}%"
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device, epoch, phase='Val'):
    """Validate model"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc=f"Epoch {epoch+1} [{phase}]")
        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({
                'loss': f"{running_loss/(batch_idx+1):.4f}",
                'acc': f"{100.*correct/total:.2f}%"
            })
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

print("✓ Training functions defined")

## 12. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'lr': []
}

# Best model tracking
best_val_acc = 0.0
best_epoch = 0
patience_counter = 0

print("\n" + "="*80)
print(f"🚀 Starting Training: Super_Mamba on PlantVillage")
print("="*80)
print(f"Epochs: {CONFIG['num_epochs']} | Batch size: {CONFIG['batch_size']} | LR: {CONFIG['learning_rate']}")
print(f"Device: {device} | Early stopping patience: {CONFIG['patience']}")
print("="*80 + "\n")

start_time = time.time()

for epoch in range(CONFIG['num_epochs']):
    # Train
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, device, epoch
    )
    
    # Validate
    val_loss, val_acc = validate(
        model, val_loader, criterion, device, epoch, phase='Val'
    )
    
    # Update learning rate
    if CONFIG['scheduler'] == 'cosine':
        scheduler.step()
    else:
        scheduler.step(val_acc)
    
    current_lr = optimizer.param_groups[0]['lr']
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    # Print epoch summary
    print(f"\nEpoch {epoch+1}/{CONFIG['num_epochs']} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"  LR: {current_lr:.6f}")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
        patience_counter = 0
        
        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'config': CONFIG
        }
        
        torch.save(checkpoint, CONFIG['save_dir'] / 'super_mamba_best.pth')
        print(f"  ✓ New best model saved! Val Acc: {val_acc:.2f}%")
    else:
        patience_counter += 1
        print(f"  No improvement ({patience_counter}/{CONFIG['patience']})")
    
    # Early stopping
    if patience_counter >= CONFIG['patience']:
        print(f"\n⚠️ Early stopping triggered after {epoch+1} epochs")
        print(f"Best Val Acc: {best_val_acc:.2f}% at epoch {best_epoch}")
        break
    
    print("-" * 80)

# Training complete
elapsed_time = time.time() - start_time
print("\n" + "="*80)
print("✓ Training Complete!")
print("="*80)
print(f"Total time: {elapsed_time/3600:.2f} hours")
print(f"Best Val Acc: {best_val_acc:.2f}% at epoch {best_epoch}")
print(f"Model saved to: {CONFIG['save_dir'] / 'super_mamba_best.pth'}")
print("="*80)

## 13. Save Final Model

In [ ]:
# Save final model
final_checkpoint = {
    'epoch': epoch + 1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'val_acc': val_acc,
    'val_loss': val_loss,
    'history': history,
    'config': CONFIG
}

torch.save(final_checkpoint, CONFIG['save_dir'] / 'super_mamba_final.pth')
print(f"✓ Final model saved to: {CONFIG['save_dir'] / 'super_mamba_final.pth'}")

## 14. Plot Training History

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Acc', linewidth=2)
axes[1].plot(history['val_acc'], label='Val Acc', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training & Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Learning Rate
axes[2].plot(history['lr'], linewidth=2, color='green')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Learning Rate')
axes[2].set_title('Learning Rate Schedule')
axes[2].set_yscale('log')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(CONFIG['save_dir'] / 'training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Training curves saved to: {CONFIG['save_dir'] / 'training_history.png'}")

## 15. Load Best Model & Test

In [ ]:
# Load best model
print("Loading best model for testing...")
checkpoint = torch.load(CONFIG['save_dir'] / 'super_mamba_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✓ Loaded best model from epoch {checkpoint['epoch']}")
print(f"  Val Acc: {checkpoint['val_acc']:.2f}%")

In [ ]:
# Test evaluation
print("\n" + "="*80)
print("🧪 Testing on Test Set")
print("="*80)

model.eval()
test_loss = 0.0
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    pbar = tqdm(test_loader, desc="Testing")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        pbar.set_postfix({
            'acc': f"{100.*correct/total:.2f}%"
        })

test_loss /= len(test_loader)
test_acc = 100. * correct / total

print("\n" + "="*80)
print("📊 Test Results:")
print("="*80)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")
print(f"Correct: {correct:,} / {total:,}")
print("="*80)

## 16. Detailed Classification Report

In [ ]:
# Classification report
from sklearn.metrics import classification_report

print("\n📋 Classification Report:\n")
print(classification_report(
    all_labels, 
    all_preds, 
    target_names=class_names,
    digits=4
))

## 17. Confusion Matrix

In [ ]:
# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(20, 18))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - Super_Mamba on PlantVillage', fontsize=16, pad=20)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(CONFIG['save_dir'] / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Confusion matrix saved to: {CONFIG['save_dir'] / 'confusion_matrix.png'}")

## 18. Save Results Summary

In [ ]:
# Save results to JSON
import json

results = {
    'model_name': 'Super_Mamba',
    'dataset': 'PlantVillage',
    'num_classes': CONFIG['num_classes'],
    'total_params': total_params,
    'best_epoch': best_epoch,
    'best_val_acc': best_val_acc,
    'test_acc': test_acc,
    'test_loss': test_loss,
    'training_time_hours': elapsed_time / 3600,
    'config': {k: str(v) if isinstance(v, Path) else v for k, v in CONFIG.items()}
}

with open(CONFIG['save_dir'] / 'results_summary.json', 'w') as f:
    json.dump(results, f, indent=4)

print(f"✓ Results saved to: {CONFIG['save_dir'] / 'results_summary.json'}")
print("\n✅ All done! MambaTSR training completed successfully.")